https://data-science-learning.com/archives/1085

In [1]:
import os
from glob import glob
import pandas as pd
import linecache
from sklearn.metrics import classification_report

# ０1 データセットの準備

In [2]:
categories = glob('./text/*')
categories = [os.path.basename(x) for x in categories if not x.endswith('.txt')]
categories

['movie-enter',
 'it-life-hack',
 'kaden-channel',
 'topic-news',
 'livedoor-homme',
 'peachy',
 'sports-watch',
 'dokujo-tsushin',
 'smax']

In [3]:
# カテゴリをid化する辞書
cat2id = {v:i for i, v in enumerate(categories)}
cat2id

{'movie-enter': 0,
 'it-life-hack': 1,
 'kaden-channel': 2,
 'topic-news': 3,
 'livedoor-homme': 4,
 'peachy': 5,
 'sports-watch': 6,
 'dokujo-tsushin': 7,
 'smax': 8}

In [4]:
def file2text(file):
    with open(file, 'r') as f:
        lines = f.readlines()

    text = ''
    
    for line in lines[2:]:
        text += line.replace('\n', '')

    return text

## 01-1 データをデータフレームに入れる

In [70]:

data_dic = {
    'cat': [],
    'text': [],
    # 'cat_id': []
}
                       
for cat in categories:
    print(cat)
    files = glob(f'./text/{cat}/*.txt')
    for i, file in enumerate(files):
        data_dic['cat'].append(cat)
        data_dic['text'].append(file2text(file))
        # data_dic['label'].append(cat2id[cat])

dataset_df = pd.DataFrame(data_dic)
dataset_df = dataset_df[:200]


movie-enter
it-life-hack
kaden-channel
topic-news
livedoor-homme
peachy
sports-watch
dokujo-tsushin
smax


In [71]:
dataset_df['label'] = dataset_df['cat'].map(cat2id)
dataset_df = dataset_df[['text', 'label']]
dataset_df

,text,label
0,【DVDエンター！】誘拐犯に育てられた女が目にした真実は、孤独か幸福か 2005年11月から...,0
1,藤原竜也、中学生とともにロケット打ち上げに成功 「アンテナを張りながら生活をしていけばいい」...,0
2,『戦火の馬』ロイヤル・プレミアにウィリアム王子＆キャサリン妃が出席 3月2日より全国ロードシ...,0
3,香里奈、女子高生100人のガチンコ質問に回答「ラーメンも食べる」 女優の香里奈が18日、都内...,0
4,ユージの前に立ちはだかったJOY「僕はAKBの高橋みなみを守る」 5日、東京・千代田区の内幸...,0
...,...,...
195,世界各国で話題沸騰！“人類の起源”を解き明かす謎の手がかり ダーウィンは1859年に発表した...,0
196,トム・クルーズがロックの神様に！ 絶大な人気を誇るミュージカルが映画化 トムがイェーーイ！！...,0
197,インタビュー：ポール・ハギス監督「自分の価値は、証明し続けなければならない」ラッセル・クロウ...,0
198,悪魔も恐れるあの“燃焼系”ヒーローが還ってきた！ニコラス・ケイジ主演『ゴーストライダー2』公...,0


## 01-2 datesets形式に変換

In [72]:
from datasets import Dataset

In [73]:
dataset = Dataset.from_pandas(dataset_df)

In [74]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 200
})

## 01-3 データセットにトークン化したデータを追加

In [75]:
from transformers import AutoTokenizer

In [76]:
tokenizer= AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-v2')

In [77]:
tokenizer('今日は暑かった')

{'input_ids': [2, 13711, 897, 2778, 11191, 881, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [78]:
def preprocess_function(examples):
    MAX_LENGTH = 512
    return tokenizer(examples["text"], max_length=MAX_LENGTH, truncation=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 200
})

In [ ]:
# tokenized_dataset = tokenized_dataset.remove_columns(['cat', 'column_to_remove2'])

In [52]:
# splited_dataset = tokenized_dataset.train_test_split(test_size=0.2)
# splited_dataset

In [20]:
# splited_dataset['train']['text'][0]

'アルインコ、イスにセットして使う「どこでもマッサージャー\u3000モミっくすキングＮＥＯ」を発売【売れ筋チェック】アルインコ株式会社は、イスにセットするだけで、本格マッサージが楽しめる「どこでもマッサージャー\u3000モミっくすキングＮＥＯ」９月１日より全国のホームセンターや電気店・家電量販店などで発売する。■普段使用しているイスに取り付けるだけでマッサージャーに！「どこでもマッサージャー\u3000モミっくすキングＮＥＯ」は、ご自宅のリビングやデスク、オフィスチェアなど、普段の生活で使用しているイスに取り付けるだけで本格的なマッサージャーになる持ち運びタイプの器具。肩エアーマッサージ、腰エアーマッサージ、もみ玉による腰もみマッサージの３種類のメニューに加え、座面シートには、ぶるぶる震えて下半身をリフレッシュさせる振動マッサージ機能が搭載されている。腰のもみ玉はヒーター内蔵なので、寒い日の温感マッサージも可能。冷えなどで凝り固まった筋肉のもみほぐしと血行促進におすすめだという。リモコン付きなので座ったまま姿勢を変えずに操作可能。プログラムコースも２種類あり、ワンタッチ操作で肩から腰まで手軽にもみほぐしてくれる。本体は折り畳み式なので、使用しないときはお部屋の隅に収納でき、椅子型のマッサージャーのようにスペースを取るということはない。多機能で、どこでも使えて、場所も取らない画期的なマッサージ器。使い方はあなた次第。オフィスで、リビングで、寝室で、ライフスタイルに合わせて自在に活用できるのだ。'

In [53]:
# splited_dataset['train']['cat_id'][0]

# 02 modelの用意

In [79]:
from transformers import AutoTokenizer

tokenizer= AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-v2')


In [80]:

def preprocess_function(examples):
    MAX_LENGTH = 512
    return tokenizer(examples["text"], max_length=MAX_LENGTH, truncation=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [81]:
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.2)
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 160
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 40
    })
})

In [82]:
tokenized_dataset['train']

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 160
})

# datacollatorについて調べること

In [92]:
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, BertForSequenceClassification

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained("cl-tohoku/bert-base-japanese-v2", num_labels=len(categories))
# model = BertForSequenceClassification.from_pretrained("cl-tohoku/bert-base-japanese-v2", num_labels=len(categories))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 03 学習

## 03-1 学習準備

In [85]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {'accuracy':acc, 'f1':f1}

In [95]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    # evaluation_strategy='epoch',
    eval_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=1,
    learning_rate=2e-5,
    use_cpu=True, # GPUを使用する場合はFalse
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)


## 03-2 学習

In [96]:

trainer.train()

  0%|          | 0/100 [00:00<?, ?it/s]

{'loss': 0.5607, 'grad_norm': 0.8691719174385071, 'learning_rate': 1.6000000000000003e-05, 'epoch': 1.0}


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 0.03430939465761185, 'eval_accuracy': 1.0, 'eval_f1': 1.0, 'eval_runtime': 7.4645, 'eval_samples_per_second': 5.359, 'eval_steps_per_second': 0.67, 'epoch': 1.0}


ValueError: You are trying to save a non contiguous tensor: `bert.encoder.layer.0.attention.self.query.weight` which is not allowed. It either means you are trying to save tensors which are reference of each other in which case it's recommended to save only the full tensors, and reslice at load time, or simply call `.contiguous()` on your tensor to pack it before saving.

In [ ]:
trainer.save_state()
trainer.save_model()

# 04 評価

In [ ]:
pred_result = trainer.predict(tokenized_dataset['test'], ignore_keys=['loss', 'last_hidden_state', 'hidden_states', 'attentions'])
pred_label= pred_result.predictions.argmax(axis=1).tolist()
print(pred_label)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(tokenized_dataset['test']['label'], pred_label, target_names=categories))